In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "1"  # Set the GPUs to use
import torch
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

Current cuda device: 0
Count of using GPUs: 1


In [2]:
import os
import re
import random
import warnings

import pandas as pd
import numpy as np
import torch

from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM, 
)
from datasets import Dataset

warnings.filterwarnings('ignore')

- Evaluation 데이터세트를 이용해 생성 메소드를 실험합니다.
- 모델의 raw output과 각종 메소드로 생성된 결과물을 비교합니다.

In [3]:
MODEL_CHECKPOINT = 'training_results/v2/ainize_kobart_news_v2/checkpoint-5070'
DATA_PATH = 'data/original/train.csv'

In [4]:
data_df = pd.read_csv(DATA_PATH)

dataset = Dataset.from_pandas(data_df).shuffle(seed=100).train_test_split(0.1)
train_dataset = dataset['train']
eval_dataset = dataset['test']

data_df = eval_dataset.to_pandas()

In [5]:
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

## Raw Output

In [6]:
idx = random.randint(0, len(data_df)-1)

sentence = data_df.context.iloc[idx]

inputs = tokenizer(sentence, max_length=1026, truncation=True, return_tensors="pt")
inputs = {k: v.cpu() for k, v in inputs.items() if k != 'token_type_ids'}
print(inputs['input_ids'].shape)

logits = model(**inputs).logits
print(logits.shape)

keys = logits.argmax(-1).squeeze().tolist()

tokenizer.decode(keys)

torch.Size([1, 97])
torch.Size([1, 97, 30000])


'<s>제16정 제1항 제169회 완주군의회 임시회 회기는결정의 건은 상정하여</s></s>69회 완주군의회 임시회 회기는 가의 건은 대해서는 운영위원회 김상식 위원장 외 3인의 의원이 발의 한대로 4월 19일부터 4월 26일까지 8일간의 일정으로 가기를 결정하고자 합니다.</s></s>들과의 이의 있으십니까?</s>『없습니다』하는 의원 있음) 가의가 없으므로 가결되었음을 선포함</s>'

# Demo of Decoding Methods

# Input

- Evaluation 데이터세트에서 무작위로 샘플을 선택합니다.

In [7]:
idx = random.randint(0, len(data_df)-1)
sample = data_df.context.iloc[idx]
y = data_df.summary.iloc[idx]
sample

'의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 회의를 속개하겠습니다. 의사일정 제4항 제7대 완주군의회 후반기 상임위원회 위원 선임의 건을 상정합니다. 완주군의회 위원회 조례 제2조에 따른 상임위원회 설치와 그 위원의 정족수는 의회운영위원회 5인 이내, 자치행정위원회 5인 이내, 산업건설위원회 5인 이내로 3개 상임위원회를 설치하도록 규정하고 있습니다. 또한, 같은 조례 제4조에 의원은 하나의 위원이 되며 다만, 의회운영위원회 위원은 겸할 수 있게 되어있습니다. 그리고 상임위원의 선임은 같은 조례 제9조제1항에 따라 의장이 추천하여 본회의 의결로 선임하도록 되어있습니다. 추천에 앞서, 본 건은 사전에 의원님들로부터 신청을 받아 의원님 한 분, 한 분의 의사를 최대한 존중하면서 합리적인 원 구성을 위해 의원님들과의 협의 등 최대한 노력하였음을 말씀드립니다. 그러면, 각 상임위원회별로 위원을 추천하겠습니다. 먼저 의회운영위원회 위원에 윤수봉 의원님, 최상철 의원님, 이인숙 의원님, 류영렬 의원님, 김용찬 의원님 이상 다섯 분을 추천하고, 다음은 자치행정위원회 위원으로는 류영렬 의원님, 최상철 의원님, 최등원 의원님, 박웅배 의원님 이상 네 분을 추천하며, 끝으로 산업건설위원회 위원으로 서남용 의원님, 윤수봉 의원님, 이향자 의원님, 김용찬 의원님, 이인숙 의원님 이상 다섯 분을 추천합니다. 미리 배부해드린 상임위원 선임안을 참고하여 주시기 바랍니다. 다른 의견이 없으시면 원안대로 의결하고자 하는데, 의원 여러분 이의 있으십니까? (“없습니다” 하는 의원 있음) 이의가 없으므로 가결되었음을 선포합니다. 각 상임위원회에서는 완주군의회 위원회 조례 제11조제2항의 규정에 따라 간사를 호선하여 본회의에 보고하여 주시기 바랍니다.'

- 모델 입력을 위한 전처리를 수행합니다.

In [8]:
max_input_length = 1026

input_ids, _, _ = tokenizer(sample, max_length=max_input_length, truncation=True, return_tensors="pt").values()
input_ids = input_ids.cpu()

## Greedy Search

- Greedy Search를 활용하여 요약문을 생성합니다.

In [9]:
outputs = model.generate(input_ids, max_length=512)
y_hat = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print('정답: ', y)
for sentence in y_hat:
    print('예측: ', sentence)

정답:  제7대 완주군의회 후반기 상임위원회의 의회운영위원회 위원으로 윤수봉 의원, 최상철 의원, 이인숙 의원, 류영렬 의원, 김용찬 의원이 선임됨. 자치행정위원회 위원으로 류영렬 의원, 최상철 의원, 최등원 의원, 박웅배 의원이 선임됨. 산업건설위원회 위원으로 서남용 의원, 윤수봉 의원, 이향자 의원, 김용찬 의원, 이인숙 의원이 선임됨.
예측:  제7대 완주군의회 후반기 상임위원회의 의회운영위원회 위원으로 윤수봉 의원, 최상철 의원, 이인숙 의원, 류영렬 의원, 김용찬 의원이 선임됨. 자치행정위원회 위원으로 류영렬 의원, 최상철 의원, 최등원 의원, 박웅배 의원이 선임됨. 산업건설위원회 위원으로 서남용 의원, 윤수봉 의원, 이향자 의원, 김용찬 의원, 이인숙 의원이 선임됨.


## Beam Search

- Beam Search를 활용해 요약문을 생성합니다.
- beam size 5에 bi-gram을 적용한 결과입니다.

- num_beams: Number of beams for beam search. 1 means no beam search.
    - (int, optional, defaults to 1)
- no_repeat_ngram_size: If set to int > 0, all ngrams of that size can only occur once.
    - (int, optional, defaults to 0)

In [10]:
outputs = model.generate(
    input_ids, max_length=512, 
    num_beams=5, 
    no_repeat_ngram_size=2, 
    num_return_sequences=5, 
    early_stopping=True,
)
y_hat = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print('정답: ', y)
for sentence in y_hat:
    print('예측: ', sentence)

정답:  제7대 완주군의회 후반기 상임위원회의 의회운영위원회 위원으로 윤수봉 의원, 최상철 의원, 이인숙 의원, 류영렬 의원, 김용찬 의원이 선임됨. 자치행정위원회 위원으로 류영렬 의원, 최상철 의원, 최등원 의원, 박웅배 의원이 선임됨. 산업건설위원회 위원으로 서남용 의원, 윤수봉 의원, 이향자 의원, 김용찬 의원, 이인숙 의원이 선임됨.
예측:  제7대 완주군의회 후반기 상임위원회의 의회운영위원회 위원으로 윤수봉 의원, 최상철 의원, 이인숙 의원, 류영렬 의원, 김용찬 의원, 박웅배 의원이 선임됨. 자치행정위원회 위원은 류철현 의원, 최등원 의원, 박성배 의원, 산업건설위원회위원은 서남용 의원 등 다섯 명의 위원을 선임함.
예측:  제7대 완주군의회 후반기 상임위원회의 의회운영위원회 위원으로 윤수봉 의원, 최상철 의원, 이인숙 의원, 류영렬 의원, 김용찬 의원, 박웅배 의원이 선임됨. 자치행정위원회 위원은 류철현 의원, 최등원 의원, 박성배 의원, 산업건설위원회위원은 서남용 의원 등 다섯 명 선임되었음.
예측:  제7대 완주군의회 후반기 상임위원회의 의회운영위원회 위원으로 윤수봉 의원, 최상철 의원, 이인숙 의원, 류영렬 의원, 김용찬 의원, 박웅배 의원이 선임됨. 자치행정위원회 위원은 류철현 의원, 최등원 의원, 박성배 의원, 산업건설위원회위원은 서남용 의원 등 다섯 명 선임함.
예측:  제7대 완주군의회 후반기 상임위원회의 의회운영위원회 위원으로 윤수봉 의원, 최상철 의원, 이인숙 의원, 류영렬 의원, 김용찬 의원, 박웅배 의원이 선임됨. 자치행정위원회 위원은 류철현 의원, 최등원 의원, 박성배 의원, 산업건설위원회위원은 서남용 의원 등 다섯 명 선임.
예측:  제7대 완주군의회 후반기 상임위원회의 의회운영위원회 위원으로 윤수봉 의원, 최상철 의원, 이인숙 의원, 류영렬 의원, 김용찬 의원, 박웅배 의원이 선임됨. 자치행정위원회 위원은 류철현 의원, 최등원 의원, 박성배 의원, 산업건설위원회위원은 서남용 의원 등 다섯 명의 위원을 선임 함.


## Sampling

- 각종 Sampling 기법을 활용해 요약문을 생성합니다.
- 우선 temperature를 조정해보고, 이어서 top-k와 top-p를 조정해봅니다.

- temperature: The value used to module the next token probabilities.
    - (float, optional, defaults to 1.0)
- top_k: The number of highest probability vocabulary tokens to keep for top-k-filtering.
    - (int, optional, defaults to 50)
- top_p: If set to float < 1, only the most probable tokens with probabilities that add up to top_p or higher are kept for generation.
    - (float, optional, defaults to 1.0)

In [11]:
outputs = model.generate(
    input_ids, max_length=512, 
    do_sample=True, 
    top_k=0,
    temperature=0.7,
)
y_hat = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print('정답: ', y)
for sentence in y_hat:
    print('예측: ', sentence)

정답:  제7대 완주군의회 후반기 상임위원회의 의회운영위원회 위원으로 윤수봉 의원, 최상철 의원, 이인숙 의원, 류영렬 의원, 김용찬 의원이 선임됨. 자치행정위원회 위원으로 류영렬 의원, 최상철 의원, 최등원 의원, 박웅배 의원이 선임됨. 산업건설위원회 위원으로 서남용 의원, 윤수봉 의원, 이향자 의원, 김용찬 의원, 이인숙 의원이 선임됨.
예측:  제7대 완주군의회 후반기 상임위원회의 의회운영위원회 위원으로 윤수봉 의원, 최상철 의원, 이인숙 의원, 류영렬 의원, 김용찬 의원이 선임됨. 자치행정위원회 위원으로 류영렬 의원, 최상철 의원, 최등원 의원, 박웅배 의원이 선임됨. 산업건설위원회 위원으로 서남용 의원, 윤수봉 의원, 이향자 의원, 김용찬 의원, 이인숙 의원이 선임됨.


### Top-K Sampling

In [12]:
outputs = model.generate(
    input_ids, max_length=512, 
    do_sample=True, 
    top_k=50,
)
y_hat = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print('정답: ', y)
for sentence in y_hat:
    print('예측: ', sentence)

정답:  제7대 완주군의회 후반기 상임위원회의 의회운영위원회 위원으로 윤수봉 의원, 최상철 의원, 이인숙 의원, 류영렬 의원, 김용찬 의원이 선임됨. 자치행정위원회 위원으로 류영렬 의원, 최상철 의원, 최등원 의원, 박웅배 의원이 선임됨. 산업건설위원회 위원으로 서남용 의원, 윤수봉 의원, 이향자 의원, 김용찬 의원, 이인숙 의원이 선임됨.
예측:  제7대 완주군의회 후반기 상임위원회 위원으로 의회운영위원회 위원 5인 이내, 자치행정위원회 5인 이내, 산업건설위원회 5인 이내로 3개 상임위원회를 설치하도록 규정함. 의회운영위원회 위원으로 윤수봉 의원, 최상철 의원, 이인숙 의원, 류영렬 의원, 김용찬 의원이 선임됨. 자치행정위원회 위원으로 류영렬 의원, 최상철 의원, 최등원 의원, 박웅배 의원이 선임됨. 산업건설위원회 위원으로 서남용 의원, 윤수봉 의원, 이향자 의원, 김용찬 의원, 이인숙 의원이 선임됨.


### Top-p (nucleus) sampling

In [13]:
outputs = model.generate(
    input_ids, max_length=512, 
    do_sample=True, 
    top_p=0.92, 
    top_k=0,
)
y_hat = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print('정답: ', y)
for sentence in y_hat:
    print('예측: ', sentence)

정답:  제7대 완주군의회 후반기 상임위원회의 의회운영위원회 위원으로 윤수봉 의원, 최상철 의원, 이인숙 의원, 류영렬 의원, 김용찬 의원이 선임됨. 자치행정위원회 위원으로 류영렬 의원, 최상철 의원, 최등원 의원, 박웅배 의원이 선임됨. 산업건설위원회 위원으로 서남용 의원, 윤수봉 의원, 이향자 의원, 김용찬 의원, 이인숙 의원이 선임됨.
예측:  제7대 완주군의회 후반기 상임위원회의 의회운영위원회 위원으로 윤수봉 의원, 최상철 의원, 이인숙 의원, 류영렬 의원, 김용찬 의원이 선임됨. 자치행정위원회 위원으로 류영렬 의원, 최상철 의원, 최등원 의원, 박웅배 의원이 선임됨. 산업건설위원회 위원으로 서남용 의원, 윤수봉 의원, 이향자 의원, 김용찬 의원, 이인숙 의원이 선임됨.


#### top_k = 50 and set top_p = 0.95 and num_return_sequences = 3

In [14]:
outputs = model.generate(
    input_ids, max_length=512, 
    do_sample=True, 
    top_k=50, 
    top_p=0.95, 
    num_return_sequences=5,
)
y_hat = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print('정답: ', y)
for sentence in y_hat:
    print('예측: ', sentence)

정답:  제7대 완주군의회 후반기 상임위원회의 의회운영위원회 위원으로 윤수봉 의원, 최상철 의원, 이인숙 의원, 류영렬 의원, 김용찬 의원이 선임됨. 자치행정위원회 위원으로 류영렬 의원, 최상철 의원, 최등원 의원, 박웅배 의원이 선임됨. 산업건설위원회 위원으로 서남용 의원, 윤수봉 의원, 이향자 의원, 김용찬 의원, 이인숙 의원이 선임됨.
예측:  제7대 완주군의회 후반기 상임위원회의 의회운영위원회 위원으로 윤수봉 의원, 최상철 의원, 이인숙 의원, 류영렬 의원, 김용찬 의원이 선임됨. 자치행정위원회 위원으로 류영렬 의원, 최상철 의원, 최등원 의원, 박웅배 의원이 선임됨. 산업건설위원회 위원으로 서남용 의원, 윤수봉 의원, 이향자 의원, 김용찬 의원, 이인숙 의원이 선임됨.
예측:  제7대 완주군의회 후반기 상임위원회 위원으로 의회운영위원회 위원 6인 중에서 윤수봉 의원, 최상철 의원, 이인숙 의원, 류영렬 의원, 김용찬 의원이 선임됨. 자치행정위원회 위원 6인 중 최상철 의원, 최등원 의원, 박웅배 의원이 선임됨. 산업건설위원회 위원으로 서남용 의원, 윤수봉 의원, 이향자 의원, 김용찬 의원, 이인숙 의원이 선임됨.
예측:  제7대 완주군의회 후반기 상임위원회의 의회운영위원회 위원으로 윤수봉 의원, 최상철 의원, 이인숙 의원, 류영렬 의원, 김용찬 의원이 선임됨. 자치행정위원회 위원으로 류영렬 의원, 최상철 의원, 최등원 의원, 박웅배 의원이 선임됨. 산업건설위원회 위원으로 서남용 의원, 윤수봉 의원, 이향자 의원, 김용찬 의원, 이인숙 의원이 선임됨.
예측:  제7대 완주군의회 후반기 상임위원회 위원으로 의회운영위원회 위원 5인 이내, 자치행정위원회 5인 이내, 산업건설위원회 5인 이내로 3개 상임위원회를 설치하기 위해 의원들을 추천함. 해당 안건은 가결됨.
예측:  제7대 완주군의회 후반기 상임위원회 위원으로 의회운영위원회 위원 5인 이내, 자치행정위원회 5인 이내, 산업건설위원회 5인 이내로 3개 상임위원회를 설치하도록 규정함. 선임됨. 자치행정위원